In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import shutil

from scipy import signal

import sys
sys.path.append('.../bats-code')

import bat_functions as bf
from CountLine import CountLine
import cv2
import matplotlib as mpl
import utm
import matplotlib.pyplot as plt

In [ ]:
plot_folder = '.../bats-data/plots'
save_folder = os.path.join(plot_folder, 'software-overview')
os.makedirs(save_folder, exist_ok=True)

In [ ]:
root_folder = ".../kasanka-bats/processed/deep-learning"
root_frame_folder = ".../Elements/bats"
date = "17Nov"
camera = "BBC"

frame_files = sorted(
    glob.glob(os.path.join(root_frame_folder,
                           date,
                           camera, 
                           "*/*.jpg"
                    )
    )
)
positions = np.load(os.path.join(root_folder, 
                                 date, camera, 
                                 'centers.npy'), 
                    allow_pickle=True)
print(len(frame_files), len(positions))

In [ ]:
shift = 0
padding = 48
frame_ind = 25000
plt.figure(figsize=(20,20))
im_name = os.path.basename(frame_files[frame_ind])
new_im_file = os.path.join(save_folder, im_name)
if not os.path.exists(new_im_file):
    copy = shutil.copy(frame_files[frame_ind], save_folder)
    print(copy)
im = cv2.imread(frame_files[frame_ind])
im_name_noext = os.path.splitext(new_im_file)[0]
depadded_im_name = im_name_noext + '-depadded.jpg'
depadded_file = os.path.join(save_folder, depadded_im_name)
if not os.path.exists(depadded_file):
    cv2.imwrite(depadded_file, im[padding:-padding, padding:-padding])

plt.imshow(im / im.max())
plt.scatter(shift + positions[frame_ind][:,0], 
            shift + positions[frame_ind][:,1], 
            s=1, c='r')

In [ ]:
tracks_file = os.path.join(root_folder, date, camera, 'raw_tracks.npy')
raw_tracks = np.load(tracks_file, allow_pickle=True)

In [ ]:
tracks_in_frame = bf.get_tracks_in_frame(frame_ind, raw_tracks)

In [ ]:
def cv2_draw_track_on_frame(frame, frame_ind, track, shift=0,
                           color=(255, 255, 255), track_width=2):
    rel_frame =  frame_ind - track['first_frame']
    if rel_frame > 0:
        for pos_ind, pos in enumerate(track['track'][1:rel_frame]):
            start_point = (int(track['track'][pos_ind, 0] + shift),
                           int(track['track'][pos_ind, 1] + shift))
            end_point = (int(pos[0] + shift), int(pos[1] + shift))
            
            cv2.line(frame, start_point, end_point, color, track_width)

In [ ]:
rel_frame

In [ ]:
track_ind = 34
border = 5

im = cv2.imread(frame_files[frame_ind])

track = tracks_in_frame[track_ind]
rel_frame = frame_ind - track['first_frame']
contour = track['contour'][rel_frame]
# cv2.drawContours(im, [contour], 0, (0,255,255), -1)

rect = cv2.minAreaRect(contour)
box = cv2.boxPoints(rect)
box_d = np.int0(box)
cv2.drawContours(im, [box_d], 0, (0,255,100), 1)
cv2.drawContours(im, [contour], 0, (255,255,100), 1)

xmin = np.min(contour[:,0]) - border
ymin = np.min(contour[:,1]) - border
xmax = np.max(contour[:,0]) + border
ymax = np.max(contour[:,1]) + border


cropped_im = im[ymin-border:ymax+border, xmin-border:xmax+border]

new_height = 500

scale = new_height / cropped_im.shape[0] 

new_width = int(cropped_im.shape[1] * scale)
dim = (new_width, new_height)

cropped_im = cv2.resize(cropped_im, dim, interpolation=cv2.INTER_NEAREST)

save_im_name = im_name_noext + 'one-track-zoom-contour-big.jpg'
save_file = os.path.join(save_folder, save_im_name)

if not os.path.exists(save_file):
    cv2.imwrite(save_file, cropped_im)
    
cropped_im_rgb = cv2.cvtColor(cropped_im, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(5, 5))
plt.imshow(cropped_im_rgb)

In [ ]:
im = cv2.imread(depadded_file)

track_inds = [track_ind]

for track in tracks_in_frame:
    rel_frame = frame_ind - track['first_frame']
    contour = track['contour'][rel_frame]
    cv2.drawContours(im, [contour-padding], 0, (0,255,255), -1)
save_im_name = im_name_noext + '-contours-one-track.jpg'
save_file = os.path.join(save_folder, save_im_name)
for track_ind in track_inds:
    cv2_draw_track_on_frame(im, frame_ind, tracks_in_frame[track_ind], 
                            shift=-48,
                            color=(255, 255, 255), track_width=2)
if not os.path.exists(save_file):
    cv2.imwrite(save_file, im)
plt.figure(figsize=(20,20))
rgb_im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_im)

In [ ]:
im = cv2.imread(depadded_file)

track_inds = np.arange(len(tracks_in_frame))

for track in tracks_in_frame:
    rel_frame = frame_ind - track['first_frame']
    contour = track['contour'][rel_frame]
#     cv2.drawContours(im, [contour-padding], 0, (0,255,255), -1)
save_im_name = im_name_noext + '-all-tracks-with-count-line.jpg'
save_file = os.path.join(save_folder, save_im_name)
for track_ind in track_inds:
    color = (255, 255, 255)
    cv2_draw_track_on_frame(im, frame_ind, tracks_in_frame[track_ind], 
                            shift=-48,
                            color=color, track_width=2)
    
count_height = int(im.shape[0] / 2)
start_point = (0, count_height)
end_point = (im.shape[1], count_height)
cv2.line(im, start_point, end_point, (0, 0, 255), 4)
if not os.path.exists(save_file):
    cv2.imwrite(save_file, im)
plt.figure(figsize=(20,20))
rgb_im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_im)

In [ ]:
track_ind = 34

track = tracks_in_frame[track_ind]
bf.add_wingbeat_info_to_tracks([track])

max_edge = track['max_edge']
plt.plot(max_edge)
plt.axhline(track['mean_wing'], c='g')
peaks = signal.find_peaks(max_edge)[0]
plt.scatter(peaks, max_edge[peaks], c='g')
plt.xlabel('frame number')
plt.ylabel('pixels')
HCONST = 1454.9 # pixels
height = bf.calculate_height(track['mean_wing'], 
                          HCONST, 
                          .8)
text_y = 38
plt.text(125, text_y, f'Height: {int(height)}m', fontsize=15)
peak_freq = track['peak_freq']
plt.text(112, text_y-3, f'Wingbeat: {peak_freq:.1f}Hz', 
         fontsize=15)


save_im_name = im_name_noext + f'track-{track_ind}-info-plot.jpg'
save_file = os.path.join(save_folder, save_im_name)
plt.savefig(save_file)

In [ ]:
root_folder = ".../kenya_drones/kasanka-bats/processed/deep-learning"
observations_root = os.path.join(root_folder, "observations")
obs_filename = f'{date}-observation-{camera}.npy'
obs_path = os.path.join(observations_root, date, obs_filename)
obs = np.load(obs_path, allow_pickle=True).item()
bat_accumulation = bf.get_bat_accumulation(obs['frames'])
# plt.axvline(frame_ind, c='r')
plt.plot(bat_accumulation)
plt.xlabel('frame number')
plt.ylabel('bats')

save_im_name = (f'bat_accumulation-{camera}.jpg')
save_file = os.path.join(save_folder, save_im_name)
plt.savefig(save_file)

In [ ]:
obs.item().keys()